<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/first_try/Notebooks/CompLingNoTrainerAllData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
import pandas as pd
df = pd.read_json("/content/HS-Brexit_train.json")

In [ ]:
df = df.transpose()
df = df.rename(columns={"hard_label": "label"})
df = df.astype({"label": int}, errors='raise')

In [ ]:
df

In [ ]:
import numpy as np
np.mean(df["label"])

0.09183673469387756

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[["text","label","soft_label"]])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/784 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [ ]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=4)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=4)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [ ]:
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [ ]:
import evaluate

#TODO do f1 with sklearn score


def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #soft_label tracks both prob and the inverse prob lol
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["label"].to(device), batch["soft_label"]["1"].to(device)
      outputs = model(input_ids=inputs, attention_mask = attention_mask, labels=hard_labels)#NllLoss
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0
      
    print(evaluate_bce_score_pytorch(model, eval_dataloader))

        
      


In [ ]:
evaluate_bce_score_pytorch(model, eval_dataloader)

# Evaluate on testset

Dev set labels are not yet released

In [ ]:
df_test = pd.read_json("/content/HS-Brexit_dev.json")
df_test = df_test.transpose()
df_test = df_test.rename(columns={"hard_label": "label"})
df_test = df_test.astype({"label": int}, errors='raise')
dataset_test = Dataset.from_pandas(df_test[["text","label","soft_label"]])
tokenized_datasets_test = dataset_test.map(tokenize_function, batched=False)
tokenized_datasets_test.set_format("torch")
test_dataloader = DataLoader(tokenized_datasets_test, batch_size=4)

In [ ]:
evaluate_bce_score_pytorch(model, test_dataloader)